# Chapter 5: Linear Independence

## 5.2 Basis
Example: **Cash flow replication**

Consider the cash flows over 3 periods, given by 3-vectors. Vectors: 
\begin{align}
  e_1 = \begin{bmatrix}
          1 \\
          0 \\
          0
        \end{bmatrix}, \space \space \space
  l_1 = \begin{bmatrix}
        1 \\
        -(1+r) \\
        0
      \end{bmatrix}, \space \space \space
  l_2 = \begin{bmatrix}
        0 \\
        1 \\
        -(1+r)
      \end{bmatrix}
\end{align}

form a basis, where $r$ is the positive per-period interest rate. 

We can replicate cash flow $c=(1,2,-3)$:

\begin{align}
  c = \alpha_1 e_1 + \alpha_2 l_1 + \alpha_3 l_2 = \alpha_1 \begin{bmatrix}
          1 \\
          0 \\
          0
        \end{bmatrix} + \alpha_2 \begin{bmatrix}
        1 \\
        -(1+r) \\
        0
      \end{bmatrix} + \alpha_3 \begin{bmatrix}
        0 \\
        1 \\
        -(1+r)
      \end{bmatrix}
\end{align}

From the third component of $c$ we get $\alpha_3 = -c_3/(1+r)$. \
From the second component of $c$ we get $\alpha_2 = -c_2/(1+r) - c_3/(1+r)^2$\
From the first component of $c$ we get $\alpha_1 = c_1 + c_2/(1+r) + c_3/(1+r)^2$

In [3]:
r = 0.05 

e1 = [1,0,0]; l1 = [1,-(1+r),0]; l2 = [0,1,-(1+r)]
c = [1,2,-3]

alpha3 = -c[3]/(1+r)
alpha2 = -c[2]/(1+r) + alpha3/(1+r)
alpha1 = c[1] - alpha2

0.18367346938775508

In [5]:
alpha1*e1 + alpha2*l1 + alpha3*l2

3-element Vector{Float64}:
  1.0
  2.0
 -3.0

## 5.3 Othonormal vectors 

**Expansion in orthonormal basis** 
Vectors: 
\begin{align}
  a_1 = \begin{bmatrix}
          0 \\
          0 \\
          -1
        \end{bmatrix},
  a_2 = \frac{1}{\sqrt2} \begin{bmatrix}
        1 \\
        1 \\
        0
      \end{bmatrix},
  a_3 = \frac{1}{\sqrt2} \begin{bmatrix}
        1 \\
        -1 \\
        0
      \end{bmatrix},
\end{align}
form an orthonormal basis, and check the expansion of $x=(1,2,3)$ in this basis: 
\begin{align}
x = (a_1^Tx)a_1 + ... +  (a_n^Tx)a_n
\end{align}


In [12]:
using LinearAlgebra
a1 = [0,0,-1]; a2 = [1,1,0]/sqrt(2); a3 = [1,-1,0]/sqrt(2)

norm(a1), norm(a2), norm(a3) # check all normalized (norm = 1)

(1.0, 0.9999999999999999, 0.9999999999999999)

In [8]:
# check all orthogonal to each other 
a1'*a2, a2'*a3, a1'*a3

(0.0, -2.2371143170757382e-17, 0.0)

In [10]:
x = [1,2,3]
# get coefficients of x in orthonormal basis 
b1 = a1'*x; b2 = a2'*x; b3 = a3'*x
b1,b2,b3

(-3, 2.1213203435596424, -0.7071067811865475)

In [11]:
# expansion of x in basis 
b1*a1+b2*a2+b3*a3

3-element Vector{Float64}:
 0.9999999999999999
 1.9999999999999996
 3.0

## 5.4 Gram-Schmidt algorithm

Determines if a list of $n$-vectors $a_1, ..., a_k$ is linearly independent.

If the vectors are linearly independent, G-S algorithm produces an orthonormal collection of vectors $q_1, ..., q_k$. 

If the vectors $a_1, ..., a_{j-1}$ are linearly independent, but $a_1, ..., a_j$ are linearly dependent, the algorithm detects this and terminates. In other words, G-S finds the first vector $a_j$ that is a linear combination of the previous vectors $a_1, ..., a_{j-1}$. 


In [13]:
function gram_schmidt(a; epsilon = 1e-10)
    q = []
    k = length(a)
    for i=1:k
        # step 1: orthogonalization 
        qtilde = a[i]
        for j=1:i-1
            qj = q[j]
            qtilde -= (qj'*a[i])*qj
        end
        # step 2: test for linear dependence 
        if norm(qtilde) < epsilon
            println("Vectors are linearly dependent")
            return q
        end 
        # step 3: normalization 
        qi = qtilde/norm(qtilde)
        push!(q, qi)
    end
    return q
end


gram_schmidt (generic function with 1 method)

In [14]:
a = [ [-1, 1, -1, 1], [-1, 3, -1, 3], [1, 3, 5, 7] ]
q = gram_schmidt(a)

3-element Vector{Any}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]
 [-0.5, -0.5, 0.5, 0.5]

In [21]:
# test orthonormality 
for qi in q
    println(norm(qi))
end

1.0
1.0
1.0


In [23]:
q[1]'*q[2], q[1]'*q[3], q[2]'*q[3]

(0.0, 0.0, 0.0)

In [24]:
# example of linear dependence 
b = [ a[1], a[2], 1.3*a[1] + 0.5*a[2] ]

q = gram_schmidt(b)

Vectors are linearly dependent


2-element Vector{Any}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]

In [25]:
# example of independence-dimension inequality 
# we know that any 3 2-vectors must be dependent -- confirm w G-S 

three_two_vectors = [ [1,1], [1,2], [-1,1] ]
q = gram_schmidt(three_two_vectors)

Vectors are linearly dependent


2-element Vector{Any}:
 [0.7071067811865475, 0.7071067811865475]
 [-0.7071067811865471, 0.7071067811865478]